In [242]:
# My jupyter path on my mac is different to pip, the easy answer is to append the path to where my packages are inported
import sys
sys.path.append("/Users/calummeikle/Documents/ClimateModelling/SatelliteImagery/env/lib/python3.9/site-packages/")
sys.path.append('/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages')

In [261]:
import matplotlib.pyplot as plt
import ee
import time
import IPython.display as disp
import folium
%matplotlib inline

In [244]:
# Trigger the authentication flow.
ee.Authenticate()
 
# Initialize the library.
ee.Initialize()

Enter verification code: 4/1AY0e-g49s6BDioG7D4Q0aYLPwJM-VcZVtMNnlrqk9SVkR07NcgHdMu9b1Bw

Successfully saved authorization token.


In [262]:
# Define a method for displaying Earth Engine image tiles to a folium map.
def add_ee_layer(self, ee_image_object, vis_params, name, show=True, opacity=1, min_zoom=0):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        show=show,
        opacity=opacity,
        min_zoom=min_zoom,
        overlay=True,
        control=True
        ).add_to(self)

# Add the Earth Engine layer method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [263]:
# A geojson for the Tokyo area taken from geojson.io
geoJSON = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              139.2572021484375,
              35.420391545750746
            ],
            [
              140.18554687499997,
              35.420391545750746
            ],
            [
              140.18554687499997,
              35.98245135784044
            ],
            [
              139.2572021484375,
              35.98245135784044
            ],
            [
              139.2572021484375,
              35.420391545750746
            ]
          ]
        ]
      }
    }
  ]
}
coords = geoJSON['features'][0]['geometry']['coordinates']
AOI = ee.Geometry.Polygon(coords)
START_DATE = '2020-08-01'
END_DATE = '2020-09-01'

In [264]:
# Get an image collection for our specified area and dates
im_coll = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(AOI)
        .filterDate(START_DATE, END_DATE))

In [265]:
# All the images taken for this geographic region in the required timeframe
acq_times = im_coll.aggregate_array('system:time_start').getInfo()
[time.strftime('%x', time.gmtime(acq_time/1000)) for acq_time in acq_times]
# What we could do here is choose a good image that has all the area we need not just a corner

['08/02/20',
 '08/05/20',
 '08/05/20',
 '08/07/20',
 '08/10/20',
 '08/10/20',
 '08/12/20',
 '08/15/20',
 '08/15/20',
 '08/17/20',
 '08/20/20',
 '08/20/20',
 '08/22/20',
 '08/25/20',
 '08/25/20',
 '08/27/20',
 '08/30/20',
 '08/30/20']

In [269]:
im_coll.size().getInfo()

18

In [270]:
# Creating a list of the collections by dates
im_list = im_coll.toList(im_coll.size())

Using the formulas on this website: https://giscrack.com/list-of-spectral-indices-for-sentinel-and-landsat/ I am going to try and plot different ratios onto a folium map

In [292]:
def ndvi(img):
    """
    High NDVI values correspond to areas that reflect more in the near-infrared spectrum. 
    Higher reflectance in the near-infrared correspond to denser and healthier vegetation
    
    Formula of NDVI = (NIR – Red) / (NIR + Red)
    NDVI (Sentinel 2) = (B8 – B4) / (B8 + B4)
    """
    return img.normalizedDifference(['B8', 'B4'])

def gndvi(img):
    """
    Green Normalized Difference Vegetation Index (GNDVI) is modified version of NDVI 
    to be more sensitive to the variation of chlorophyll content in the crop.
    
    Formula of GNDVI = (NIR-GREEN) /(NIR+GREEN)
    GNDVI (Sentinel 2) = (B8 – B3) / (B8 + B3)
    """
    return img.normalizedDifference(['B8', 'B3'])

def evi(img):
    """
    EVI is similar to Normalized Difference Vegetation Index (NDVI) and can be used to quantify vegetation greenness. 
    However, EVI corrects for some atmospheric conditions and canopy background noise 
    and is more sensitive in areas with dense vegetation.
    
    Formula of EVI = G * ((NIR – R) / (NIR + C1 * R – C2 * B + L))
    EVI (Sentinel 2) = 2.5 * ((B8 – B4) / (B8 + 6 * B4 – 7.5 * B2 + 1))
    """
    param = ee.Image(2.5)
    canopy_adjustment = ee.Image(1)
    c1 = ee.Image(6)
    c2 = ee.Image(7.5)
    red_img = img.select('B4')
    green_img = img.select('B3')
    blue_img = img.select('B2')
    nir_img = img.select('B8')
    numerator = param.multiply((nir_img.subtract(red_img)))
    red_adj = c1.multiply(red_img)
    blue_adj = c2.multiply(blue_img)
    denominator = nir_img.add(red_adj).subtract(blue_adj).add(canopy_adjustment)
    return numerator.divide(denominator)


ndvi_img = ndvi(ee.Image(im_list.get(1)).clip(AOI))
gndvi_img = gndvi(ee.Image(im_list.get(1)).clip(AOI))
evi_img = evi(ee.Image(im_list.get(1)).clip(AOI))

In [293]:
evi_img

In [294]:
# url = red_img.getThumbURL({'min': 0, 'max': 2500})
# disp.Image(url=url, width=400)

In [295]:
center = AOI.centroid(10).coordinates().reverse().getInfo()
m = folium.Map(location=center, zoom_start=10)

# Add layers to the map
m.add_ee_layer(red_img, {'min': 0, 'max': 2500}, 'Red')
m.add_ee_layer(nir_img, {'min': 0, 'max': 2500}, 'NIR')
m.add_ee_layer(ndvi_img, {'min': 0, 'max': 1, 'palette': ['grey','green']}, 'NDVI') 
# the vis_param's make a differnece on the percentage of iridiance counts as vegetation so they are quite important
m.add_ee_layer(gndvi_img, {'min': 0, 'max': 1, 'palette': ['grey','green']}, 'Green NDVI')
m.add_ee_layer(evi_img, {'min': 0, 'max': 1, 'palette': ['grey','green']}, 'EVI')
# Add a layer control panel to the map.
m.add_child(folium.LayerControl())

# Display the map.
display(m)

In [274]:
# TODO: use the cloud mask method on these images, because light areas can be clouds

Now can I do something similar on an area of Senegal that has seen development of the Great Green Wall to look at the Vegetation Index and possibly Soil Index

It is hard to find any information of where the each tree is being planted. This website lists the areas of interest
https://gef6.globelegislators.org/senegal/the-great-green-wall-experience-in-senegal, I am going to focus on the matam region.

In [175]:
geoJSON = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -15.868377685546873,
              15.884734325453612
            ],
            [
              -15.582733154296875,
              15.884734325453612
            ],
            [
              -15.582733154296875,
              16.155325758629743
            ],
            [
              -15.868377685546873,
              16.155325758629743
            ],
            [
              -15.868377685546873,
              15.884734325453612
            ]
          ]
        ]
      }
    }
  ]
}

In [176]:
coords = geoJSON['features'][0]['geometry']['coordinates']
AOI = ee.Geometry.Polygon(coords)
ORIGINAL_START_DATE = '2018-12-01' 
ORIGINAL_END_DATE = '2018-12-31'

In [177]:
# Get an image collection for our specified area and dates
im_coll = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(AOI)
        .filterDate(ORIGINAL_START_DATE, ORIGINAL_END_DATE))

In [178]:
# All the images taken for this geographic region in the required timeframe
acq_times = im_coll.aggregate_array('system:time_start').getInfo()
[time.strftime('%x', time.gmtime(acq_time/1000)) for acq_time in acq_times]
# What we could do here is choose a good image that has all the area we need not just a corner

['12/14/18',
 '12/14/18',
 '12/16/18',
 '12/19/18',
 '12/19/18',
 '12/21/18',
 '12/21/18',
 '12/24/18',
 '12/24/18',
 '12/26/18',
 '12/29/18',
 '12/29/18']

In [204]:
# Creating a list of the collections by dates
im_list = im_coll.toList(im_coll.size())
# Getting the nth date in the list and then selecting the specific sentinel epectral band
blue_img = ee.Image(im_list.get(-1)).select('B2').clip(AOI)
green_img = ee.Image(im_list.get(-1)).select('B3').clip(AOI)
red_img = ee.Image(im_list.get(-1)).select('B4').clip(AOI)
nir_img = ee.Image(im_list.get(-1)).select('B8').clip(AOI)

In [234]:
# Creating a dataframe of the image data
import pandas as pd
df = pd.DataFrame(im_list.getInfo())
acq_times = im_coll.aggregate_array('system:time_start').getInfo()
df["date"] = [time.strftime('%x', time.gmtime(acq_time/1000)) for acq_time in acq_times]

In [235]:
df.set_index("date", inplace=True)
df.head() # It's ok to visualise the data, but need to find a way to read an image from this formate

,type,bands,id,version,properties
date,,,,,
12/14/18,Image,"[{'id': 'B1', 'data_type': {'type': 'PixelType...",COPERNICUS/S2_SR/20181214T113459_20181214T1140...,1557088287937734,{'DATATAKE_IDENTIFIER': 'GS2B_20181214T113459_...
12/14/18,Image,"[{'id': 'B1', 'data_type': {'type': 'PixelType...",COPERNICUS/S2_SR/20181214T113459_20181214T1140...,1557016256881882,{'DATATAKE_IDENTIFIER': 'GS2B_20181214T113459_...
12/16/18,Image,"[{'id': 'B1', 'data_type': {'type': 'PixelType...",COPERNICUS/S2_SR/20181216T112451_20181216T1131...,1557018629019192,{'DATATAKE_IDENTIFIER': 'GS2A_20181216T112451_...
12/19/18,Image,"[{'id': 'B1', 'data_type': {'type': 'PixelType...",COPERNICUS/S2_SR/20181219T113501_20181219T1140...,1563389903141111,{'DATATAKE_IDENTIFIER': 'GS2A_20181219T113501_...
12/19/18,Image,"[{'id': 'B1', 'data_type': {'type': 'PixelType...",COPERNICUS/S2_SR/20181219T113501_20181219T1140...,1563565635037021,{'DATATAKE_IDENTIFIER': 'GS2A_20181219T113501_...


In [205]:
url = red_img.getThumbURL({'min': 0, 'max': 2500})
disp.Image(url=url, width=400)

In [206]:
ndvi_img = (nir_img.subtract(red_img)).divide(nir_img.add(red_img))

In [208]:
center = AOI.centroid(10).coordinates().reverse().getInfo()
m = folium.Map(location=center, zoom_start=9)

# Add layers to the map
m.add_ee_layer(ee.Image(im_list.get(-1)), {'bands': ["B4", "B3", "B2"], 'min': 0, 'max': 2500}, 'True Colour Image')
m.add_ee_layer(red_img, {'min': 0, 'max': 2500}, 'Red')
m.add_ee_layer(nir_img, {'min': 0, 'max': 2500}, 'NIR')
m.add_ee_layer(ndvi_img, {'min': 0, 'max': 1, 'palette': ['grey','green']}, 'NDVI') 
# # the vis_param's make a differnece on the percentage of iridiance counts as vegetation so they are quite important
# m.add_ee_layer(gndvi_img, {'min': 0, 'max': 0.4, 'palette': ['grey','green']}, 'Green NDVI')
# Add a layer control panel to the map.
m.add_child(folium.LayerControl())

# Display the map.
display(m)

This is very tricky, especially with Sentinel 2.

Limitations:
* No images on GEE before Dec 2018, and if I tried to get it from the Sentinel API the satellite only started talking pictures in 2015.
* I would need a lot of images. Because I am not sure where the Great Green Wall is.
* Sentinel 2 needs Cloud Masking on every image.

Will go to Sentinel 1.